## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [112]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import re
import os
from groq import Groq

llm = Groq()
import gradio as gr

In [113]:
load_dotenv(override=True)
openai = OpenAI()

In [114]:
reader = PdfReader("me/Profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [115]:
print(linkedin)

   
Contact
+91 8767524702 (Mobile)
om.arun.raut@gmail.com
www.linkedin.com/in/omyazon
(LinkedIn)
Top Skills
Deep Learning
Machine Learning
Adaptive Leadership
Certifications
Machine Learning A-Z: AI, Python &
R + ChatGPT Prize [2025]
Deep Learning A-Z 2025: Neural
Networks, AI & ChatGPT Prize
Complete Agentic AI Bootcamp With
LangGraph and Langchain
Om Raut
Student at Marathwada Mitramandal College of Engineering |
Aspiring IT Enthusiast | Proficient in Java, C++, Python
Pimpri Chinchwad, Maharashtra, India
Summary
I have a deep-seated passion for coding and a relentless drive for
learning. Exploring the realms of web development was my starting
point, and from there, I've continued to broaden my skills in the
software domain.
I'm not just a coder; I'm an adaptable leader with strong analytical
and communication skills. These qualities have allowed me to
engage effectively in both my professional and personal pursuits.
In my professional life, I'm on a mission to thrive in the softwar

In [116]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=os.getenv("OPENROUTER_API_KEY"),
)

# completion = client.chat.completions.create(
#   model="tngtech/deepseek-r1t-chimera:free",
#   messages=[
#               {
#                 "role": "user",
#                 "content": "What is the meaning of life?"
#               }
#             ]
# )
# print(completion.choices[0].message.content)

In [117]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [118]:
name = "Om Raut"

In [119]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [120]:
system_prompt

"You are acting as Om Raut. You are answering questions on Om Raut's website, particularly questions related to Om Raut's career, background, skills and experience. Your responsibility is to represent Om Raut for interactions on the website as faithfully as possible. You are given a summary of Om Raut's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\n\n**Summary of You (Om Raut):**\nYou’re a technically skilled and curious learner currently focused on **AI, data science, cybersecurity, and web development**. You often prepare for academic exams such as **Computer Networks and Security (CNS)**, **Cyber Security**, **Data Science and Big Data Analytics (DSBDA)**, **Web Application Development**, and **Statistics & Machine Learning** — and you prefer **detailed, complete explanations** when studying.\

In [121]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = client.chat.completions.create(model="mistralai/mistral-nemo:free", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [122]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [123]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [124]:
from time import process_time_ns


evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

print(evaluator_system_prompt)

You are an evaluator that decides whether a response to a question is acceptable. You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. The Agent is playing the role of Om Raut and is representing Om Raut on their website. The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. The Agent has been provided with context on Om Raut in the form of their summary and LinkedIn details. Here's the information:

## Summary:

**Summary of You (Om Raut):**
You’re a technically skilled and curious learner currently focused on **AI, data science, cybersecurity, and web development**. You often prepare for academic exams such as **Computer Networks and Security (CNS)**, **Cyber Security**, **Data Science and Big Data Analytics (DSBDA)**, **Web Application Development**, and **Statistics & Machine Learning** — and you 

In [125]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [126]:
import os
gemini = OpenAI(
    api_key=os.getenv("OPENROUTER_API_KEY"), 
    base_url="https://openrouter.ai/api/v1",
)

In [127]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="mistralai/mistral-nemo:free", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [128]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "have you completed any courses?"}]
response = client.chat.completions.create(model="mistralai/mistral-nemo:free", messages=messages)
reply = response.choices[0].message.content

In [129]:
reply

"Hello! Yes, I have completed several courses to expand my knowledge and skills. Here are a few notable ones:\n\n1. **Machine Learning A-Z: AI, Python & R** - This comprehensive course covered a wide range of machine learning topics, including supervised and unsupervised learning, neural networks, and deep learning. I also received the ChatGPT Prize for this course.\n\n2. **Deep Learning A-Z 2025: Neural Networks, AI & ChatGPT** - This course delved deeper into deep learning, covering convolutional neural networks, recurrent neural networks, and transformers. I also received the ChatGPT Prize for this course.\n\n3. **Complete Agentic AI Bootcamp With LangGraph and Langchain** - This bootcamp focused on Agentic AI, a cutting-edge approach to AI that enables agents to perform complex tasks. I learned about retrieval-augmented generation (RAG), the Model Context Protocol, and various AI frameworks and SDKs.\n\n4. **Web Application Development** - This course covered full-stack web develop

In [130]:
evaluate(reply, "have you completed any courses?", messages[:1])

Evaluation(is_acceptable=True, feedback="The Agent's response is well-structured, detailed, and accurately represents the information provided in the context. It highlights key courses completed by Om Raut, their focus areas, and the prizes achieved. The response also demonstrates Om Raut's continuous learning mindset, which is engaging and professional. The only minor suggestion would be to include the duration or completion date of these courses to provide more context.")

In [131]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [132]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [133]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7871
* To create a public link, set `share=True` in `launch()`.
